This script allows us to use the **Random Forest** machine learning technique to classify ten states of late blight severity. This script was executed using google colab. The dzetsaka: Classification Tool [1] was used as a reference. 


First, it is neccesary to mount our drive folder, where our data imput is saved.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

It is necesary to import all the libraries and files needed to run the script. 

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/cip/')

In [3]:
import os
import numpy as np
import pickle
from osgeo import (gdal, ogr)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import accuracy_index as ai
import proccessing_algo
from sklearn.metrics import confusion_matrix, mean_squared_error,accuracy_score,cohen_kappa_score,classification_report
import pandas as pd
from sklearn import tree

In [4]:
#Loading Raster and Vector File with the classes
inRaster =['drive/MyDrive/cip/data/20191129_40M_RGBNRE_CUTSP_mergepca.TIF',
            'drive/MyDrive/cip/data/20191218_40m_RGBNRE_CUTSP_mergepca.tif',
            'drive/MyDrive/cip/data/20191129_50m_RGBNRE_mergepca.tif',
            'drive/MyDrive/cip/data/20191218_50m_RGBNRE_mergepca.tif']
inVector = ['drive/MyDrive/cip/data/20191129_40M_RGBNRE_CUTSP_seg_TRAIN_train10.shp',
            'drive/MyDrive/cip/data/20191218_40m_RGBNRE_CUTSP_TRAIN_train10.shp',
            'drive/MyDrive/cip/data/20191129_50m_RGBNRE_seg_train10.shp',
            'drive/MyDrive/cip/data/20191218_50m_RGBNRE_seg_train10.shp']
inField ='CLASS_G'
inSeed = 0
inClassifier = 'RF'


In [7]:
X = np.array([]).reshape(0, 12)# 12 : Feature Number 
Y = np.array([],dtype=np.uint16).reshape(0, 1)

In [ ]:
#Reading the Raster and Vector File
for i in range(0,4):
#Rasterize the vector file into a raster
    ROI = proccessing_algo.rasterize(inRaster[i], inVector[i], inField)
#Extracting the sample data(X) and its label(Y)
    Xi, Yi = proccessing_algo.get_samples_from_roi(inRaster[i], ROI)
    #print(i)
    #print(Xi[100,0])
    #print(Yi[100,0])
    X = np.append(X, Xi, axis=0)
    Y = np.append(Y, Yi, axis=0)
    

In [9]:
[n, d] = X.shape
#Number of classes
C = int(Y.max()) 
#Scaling all the sample data. Return the Sample scaled, the vector of the max and min value for each feature
X, M, m = proccessing_algo.scale(X)

In [ ]:
#Splitting the samples into 2/3rd for training and 1/3rd for testing
SPLIT=67
#Trainig set
x = np.array([]).reshape(0, d)
y = np.array([]).reshape(0, 1)
#Testing set
xt = np.array([]).reshape(0, d)
yt = np.array([]).reshape(0, 1)
np.random.seed(inSeed)  
for i in range(C):
    t = np.where((i + 1) == Y)[0]
    nc = t.size
    #print(nc)
    ns = int(nc * (SPLIT / float(100)))
    rp = np.random.permutation(nc)
    x = np.concatenate((X[t[rp[0:ns]], :], x))
    xt = np.concatenate((X[t[rp[ns:]], :], xt))
    y = np.concatenate((Y[t[rp[0:ns]]], y))
    yt = np.concatenate((Y[t[rp[ns:]]], yt))

In [12]:

param_grid = dict(n_estimators=3**np.arange(1, 5), max_features=range(1, x.shape[1], int( np.sqrt(x.shape[1]))))
classifier = RandomForestClassifier()
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits) 
y.shape = (y.size,)
grid = GridSearchCV(classifier,param_grid=param_grid,cv=cv)
grid.fit(x, y)
model = grid.best_estimator_
model.fit(x, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=13,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=81,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
grid.best_params_

In [ ]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(x.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

In [ ]:
model.score(x, y)

In [16]:
yp = model.predict(xt)

In [ ]:
from sklearn.metrics import confusion_matrix, mean_squared_error,accuracy_score,cohen_kappa_score,classification_report
import pandas as pd
cm = pd.DataFrame(confusion_matrix(yt, yp).T, 
                  index = ['1','2','3','4','5','6','7','8','9','10','11'], 
                  columns = ['1','2','3','4','5','6','7','8','9','10','11'])
print("Confusion Matrix")
print(cm)
accuracy_score(yt,yp)


In [ ]:
cohen_kappa_score(yt,yp)

In [ ]:
target_names =['1','2','3','4','5','6','7','8','9','10','11']
print("Classification Report")
print(classification_report(yt,yp,target_names=target_names,digits=4))

In [19]:
output_log_file = os.path.join('drive/MyDrive/cip/output/'+ 'report_5bands_nred_ndvi_5textures11.txt')

In [27]:
#Saving classification report
textf = open(output_log_file, "w")
print('Classification Report', file=textf)
print('\n', file=textf)
print('Confusion Matrix:', file=textf)
print(cm, file=textf)
print('\n', file=textf)
print('Classification Report', file=textf)
print(classification_report(yt,yp,target_names=target_names,digits=4),file=textf)
print('\n', file=textf)
print('Cohens Kappa coefficient:', file=textf)
print(cohen_kappa_score(yt,yp),file=textf)
print('\n', file=textf)
print('Feature ranking::', file=textf)
for f in range(x.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]),file=textf )

textf.close()

In [22]:
#saveConfusionMatrix
np.savetxt('drive/MyDrive/cip/output/confusion_matrix_5bands_nred_ndvi_5texturespca11.txt',cm,delimiter=',',header='Columns=prediction,Lines=reference.',fmt='%1.4d')
#saveModel
output = open('drive/MyDrive/cip/output/model_5bands_nred_ndvi_5texturespca11.txt', 'wb')
pickle.dump([model, M, m, 'RF'], output)
output.close()

In [20]:
#Load Model
modelfile = open('drive/MyDrive/cip/output/model_5bands_nred_ndvi_5texturespca11.txt', 'rb')
model, M, m, classifier = pickle.load(modelfile)
modelfile.close()

In [38]:
#Applying the Model to a Raster File
raster = gdal.Open(inRaster[3], gdal.GA_ReadOnly)
d = raster.RasterCount
nc = raster.RasterXSize
nl = raster.RasterYSize
GeoTransform = raster.GetGeoTransform()
Projection = raster.GetProjection()

band = raster.GetRasterBand(1)
block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]
del band
nodatatest = raster.GetRasterBand(1).GetNoDataValue()

In [39]:
outRaster = 'drive/MyDrive/cip/output/model_5bands_nred_ndvi_5textures_G12.tif'
driver = gdal.GetDriverByName('GTiff')
dst_ds = driver.Create(outRaster, nc, nl, 1, gdal.GDT_Byte)
dst_ds.SetGeoTransform(GeoTransform)
dst_ds.SetProjection(Projection)
out = dst_ds.GetRasterBand(1)
nClass = len(model.classes_)
# Perform the classification
total = nl * y_block_size
for i in range(0, nl, y_block_size):
    if i + y_block_size < nl:  # Check for size consistency in Y
        lines = y_block_size
    else:
         lines = nl - i
    for j in range(0, nc, x_block_size):  # Check for size consistency in X
        if j + x_block_size < nc:
            cols = x_block_size
        else:
            cols = nc - j
        X = np.empty((cols * lines, d))
        for ind in range(d):
            X[:, ind] = raster.GetRasterBand(
                int(ind + 1)).ReadAsArray(j, i, cols, lines).reshape(cols * lines)
        band_temp = raster.GetRasterBand(1)
        nodata_temp = band_temp.GetNoDataValue()
        if nodata_temp is None:
            nodata_temp = -9999
        mask_temp = band_temp.ReadAsArray(j, i, cols, lines).reshape(cols * lines)
        temp_nodata = np.where(mask_temp != nodata_temp)[0]
        t = np.where(X[:, 0] != nodata_temp)[0]
        yp = np.zeros((cols * lines,))
        if t.size > 0:
            yp[t] = model.predict(proccessing_algo.scale(X[t, :], M=M, m=m))
        out.WriteArray(yp.reshape(lines, cols), j, i)
        NODATA=0
        out.SetNoDataValue(NODATA)
        out.FlushCache()
        del X, yp
raster = None
dst_ds = None

References:
[1] Karasiak, N. (2017). Dzetsaka: Classification tool. Retrieved fromhttps://plugins.qgis.org/plugins/dzetsaka